# Importing Libraries

In [11]:
#importing python libraries
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.pipeline import FeatureUnion
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split



#rounder = FunctionTransformer(lambda array: np.round(array, decimals=2))

Typical Sklearn syntax: step_name__transformer_name__hyperparameter_name
To check which hyperparameters of the pipeline can be optimized: pipeline.get_params()

# Importing Data From All_Airports.csv file

In [12]:
dataset_b_dir = '../raw_data/Dataset_B_FORWW_Kaggle'

csv_filename = 'All_Airports.csv'

df = pd.read_csv(os.path.join(dataset_b_dir,csv_filename), compression='zip')

In [13]:
#df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15144514 entries, 0 to 15144513
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Time                    object 
 1   Origin                  object 
 2   Dest                    object 
 3   Carrier                 object 
 4   Cancelled               bool   
 5   CancellationReason      object 
 6   Delayed                 bool   
 7   DepDelayMinutes         float64
 8   CarrierDelay            float64
 9   WeatherDelay            float64
 10  NASDelay                float64
 11  SecurityDelay           float64
 12  LateAircraftDelay       float64
 13  Temperature             float64
 14  Feels_Like_Temperature  float64
 15  Altimeter_Pressure      float64
 16  Sea_Level_Pressure      float64
 17  Visibility              float64
 18  Wind_Speed              float64
 19  Wind_Gust               float64
 20  Precipitation           float64
 21  Ice_Accretion_3hr       float

In [14]:
round(df.describe(),2)

,DepDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr
count,14815623.00,3274328.00,3274328.00,3274328.00,3274328.00,3274328.00,15144514.00,15144514.00,15144514.00,15144514.00,15144514.00,15144514.00,15144514.00,15144514.00,15144514.00
mean,16.09,25.52,5.42,12.45,0.15,24.40,63.17,62.19,1016.33,1016.16,14522.99,9.07,26.09,0.17,0.00
std,51.53,67.65,34.79,28.83,3.61,56.12,18.05,21.01,6.50,6.17,30318.96,5.42,3.10,1.30,0.02
min,0.00,0.00,0.00,0.00,0.00,0.00,-22.00,-40.86,116.49,907.80,0.00,0.00,9.21,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,51.00,51.10,1012.53,1012.80,16093.40,5.75,24.74,0.00,0.00
50%,0.00,6.00,0.00,0.00,0.00,0.00,65.00,64.90,1016.26,1016.20,16093.40,8.06,26.24,0.00,0.00
75%,11.00,26.00,0.00,17.00,0.00,27.00,76.00,76.00,1020.32,1019.30,16093.40,12.66,27.62,0.00,0.00
max,3695.00,3221.00,1805.00,1409.00,1183.00,3581.00,119.00,118.60,1042.33,1068.40,33804186.70,281.94,96.67,82.30,11.94


# Taking a random sample

In [15]:
#taking a random sample to work with
#comment out the below code when we want to push the entire dataset
#through the workflow
#df = df.sample(random_state=42, weights=)

# Data Preprocessing

## Cleaning CancellationReason Column

In [16]:
print(df.CancellationReason.unique())
df[df['DepDelayMinutes'].isnull()]['Delayed'].unique()

[nan 'Weather' 'Carrier' 'Security' 'National Air System']


array([False])

In [17]:
df['CancellationReason'] = df['CancellationReason'].fillna('Not Cancelled')

In [18]:
print(df.CancellationReason.unique())

['Not Cancelled' 'Weather' 'Carrier' 'Security' 'National Air System']


## Cleaning Weather Delay Length By filling NAs with 0.0s

In [19]:
#Renaming WeatherDelay to Weather_Delay_Length
df = df.rename(columns={'WeatherDelay':'Weather_Delay_Length'})
df.head(1)

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,Weather_Delay_Length,...,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr
0,2021-01-01 09:00:00,LAX,JFK,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,NaN,...,NaN,56.0,55.9,1020.32,1020.1,16093.4,0.0,24.97,0.0,0.0


In [20]:
df['Weather_Delay_Length'] = df['Weather_Delay_Length'].fillna(0.0)

df.head(1)

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,Weather_Delay_Length,...,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr
0,2021-01-01 09:00:00,LAX,JFK,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,NaN,56.0,55.9,1020.32,1020.1,16093.4,0.0,24.97,0.0,0.0


In [21]:
df['Time'] = pd.to_datetime(df['Time'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15144514 entries, 0 to 15144513
Data columns (total 22 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Time                    datetime64[ns]
 1   Origin                  object        
 2   Dest                    object        
 3   Carrier                 object        
 4   Cancelled               bool          
 5   CancellationReason      object        
 6   Delayed                 bool          
 7   DepDelayMinutes         float64       
 8   CarrierDelay            float64       
 9   Weather_Delay_Length    float64       
 10  NASDelay                float64       
 11  SecurityDelay           float64       
 12  LateAircraftDelay       float64       
 13  Temperature             float64       
 14  Feels_Like_Temperature  float64       
 15  Altimeter_Pressure      float64       
 16  Sea_Level_Pressure      float64       
 17  Visibility              float64       
 18  

In [22]:
df.columns

Index(['Time', 'Origin', 'Dest', 'Carrier', 'Cancelled', 'CancellationReason',
       'Delayed', 'DepDelayMinutes', 'CarrierDelay', 'Weather_Delay_Length',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Temperature',
       'Feels_Like_Temperature', 'Altimeter_Pressure', 'Sea_Level_Pressure',
       'Visibility', 'Wind_Speed', 'Wind_Gust', 'Precipitation',
       'Ice_Accretion_3hr'],
      dtype='object')

# Feature Engineering WeatherDelayed

In [23]:

df['Weather_Delayed'] =  df.apply(lambda row: True if row['Weather_Delay_Length'] > 0 else False, axis=1)


In [24]:
df[df['Weather_Delay_Length'] > 0.0].head(1)

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,Weather_Delay_Length,...,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr,Weather_Delayed
395,2021-01-26,LAX,ORD,American Airlines Inc.,False,Not Cancelled,True,355.0,0.0,343.0,...,46.0,43.02,1009.14,1009.2,16093.4,5.75,24.97,0.0,0.0,True


In [25]:
df[df['Weather_Delay_Length'] == 0.0].head(1)

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,Weather_Delay_Length,...,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr,Weather_Delayed
0,2021-01-01 09:00:00,LAX,JFK,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,56.0,55.9,1020.32,1020.1,16093.4,0.0,24.97,0.0,0.0,False


# Investigating Duplicates


In [26]:
df.duplicated().sum()

np.int64(705215)

In [27]:
df[df.duplicated()].head(20)

#not real duplicates as time is different - how to tell at scale

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,Weather_Delay_Length,...,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr,Weather_Delayed
18,2021-01-08 09:00:00,LAX,JFK,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,51.0,51.1,1022.01,1015.2,9656.04,3.45,24.97,0.0,0.0,False
35,2021-01-22 09:00:00,LAX,JFK,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,57.0,57.0,1016.93,1015.2,14484.06,6.90,24.97,0.0,0.0,False
101,2021-01-08 09:00:00,LAX,HNL,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,51.0,51.1,1022.01,1015.2,9656.04,3.45,24.97,0.0,0.0,False
118,2021-01-22 09:00:00,LAX,HNL,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,57.0,57.0,1016.93,1015.2,14484.06,6.90,24.97,0.0,0.0,False
224,2021-01-22 09:00:00,LAX,KOA,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,57.0,57.0,1016.93,1015.2,14484.06,6.90,24.97,0.0,0.0,False
440,2021-01-08 09:00:00,LAX,MIA,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,51.0,51.1,1022.01,1015.2,9656.04,3.45,24.97,0.0,0.0,False
457,2021-01-22 09:00:00,LAX,MIA,American Airlines Inc.,False,Not Cancelled,True,3.0,NaN,0.0,...,57.0,57.0,1016.93,1015.2,14484.06,6.90,24.97,0.0,0.0,False
1352,2021-01-08 09:00:00,LAX,DFW,American Airlines Inc.,False,Not Cancelled,True,69.0,56.0,0.0,...,51.0,51.1,1022.01,1015.2,9656.04,3.45,24.97,0.0,0.0,False
1369,2021-01-22 09:00:00,LAX,DFW,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,57.0,57.0,1016.93,1015.2,14484.06,6.90,24.97,0.0,0.0,False
1641,2021-01-04 20:00:00,LAX,SEA,Alaska Airlines Inc.,False,Not Cancelled,False,0.0,NaN,0.0,...,57.0,57.0,1021.00,1020.8,16093.40,0.00,24.97,0.0,0.0,False


# Splitting Train and Test

In [28]:
#defining X feature df and Y target series

X = df.drop(columns=['Weather_Delayed', 'Weather_Delay_Length'])
y = df['Weather_Delayed']

In [29]:
X.head(1)

,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed,DepDelayMinutes,CarrierDelay,NASDelay,...,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,Wind_Speed,Wind_Gust,Precipitation,Ice_Accretion_3hr
0,2021-01-01 09:00:00,LAX,JFK,American Airlines Inc.,False,Not Cancelled,False,0.0,NaN,NaN,...,NaN,56.0,55.9,1020.32,1020.1,16093.4,0.0,24.97,0.0,0.0


In [30]:
y.head()

0    False
1    False
2    False
3    False
4    False
Name: Weather_Delayed, dtype: bool

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42, stratify=y)

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1514451 entries, 10921945 to 7828089
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   Time                    1514451 non-null  datetime64[ns]
 1   Origin                  1514451 non-null  object        
 2   Dest                    1514451 non-null  object        
 3   Carrier                 1514451 non-null  object        
 4   Cancelled               1514451 non-null  bool          
 5   CancellationReason      1514451 non-null  object        
 6   Delayed                 1514451 non-null  bool          
 7   DepDelayMinutes         1481705 non-null  float64       
 8   CarrierDelay            326982 non-null   float64       
 9   NASDelay                326982 non-null   float64       
 10  SecurityDelay           326982 non-null   float64       
 11  LateAircraftDelay       326982 non-null   float64       
 12  Temperature 

In [33]:
X_train.columns

Index(['Time', 'Origin', 'Dest', 'Carrier', 'Cancelled', 'CancellationReason',
       'Delayed', 'DepDelayMinutes', 'CarrierDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay', 'Temperature',
       'Feels_Like_Temperature', 'Altimeter_Pressure', 'Sea_Level_Pressure',
       'Visibility', 'Wind_Speed', 'Wind_Gust', 'Precipitation',
       'Ice_Accretion_3hr'],
      dtype='object')

# Constructing the Preprocessing Pipeline

In [34]:
num_col = make_column_selector(dtype_include=['float64'])
cat_col = make_column_selector(dtype_include=['object', 'bool'])

num_transformer = make_pipeline(SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.0), RobustScaler())
cat_transformer = OneHotEncoder(sparse_output=False, drop='if_binary')

#consider using if_binary param in ohe


In [35]:
preproc_pipe = make_column_transformer(
    (num_transformer, num_col),
    (cat_transformer, cat_col),
    remainder='passthrough'
)

#zero_imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0.0)
#df_imputed = pd.DataFrame(zero_imputer.fit_transform(df), columns=df.columns)

preproc_pipe

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value=0.0,
                                                                strategy='constant')),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x129b23cd0>),
                                ('onehotencoder',
                                 OneHotEncoder(drop='if_binary',
                                               sparse_output=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x129b22170>)])

In [38]:


X_train_preproc = pd.DataFrame(preproc_pipe.fit_transform(X_train))

X_test_preproc = pd.DataFrame(preproc_pipe.transform(X_test))

feature_names = preproc_pipe.get_feature_names_out()
feature_names = [name.split("__")[-1] for name in feature_names]

X_train_preproc.columns = feature_names

X_test_preproc.columns = feature_names

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [ ]:
X_train_preproc.head(20)

,DepDelayMinutes,CarrierDelay,NASDelay,SecurityDelay,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,...,Wind_Gust,Precipitation,Ice_Accretion_3hr,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed
0,0.0,0.0,0.0,0.0,0.0,-1.636,-2.174194,0.650834,0.907692,-9656.04,...,0.079861,0.03,0.0,2022-11-18 16:00:00,ORD,LAX,United Air Lines Inc.,False,Not Cancelled,False
1,0.0,0.0,0.0,0.0,0.0,1.24,1.614919,-0.261874,-0.338462,0.0,...,-0.479167,0.0,0.0,2022-08-09 15:00:00,BWI,SAT,Southwest Airlines Co.,False,Not Cancelled,False
2,0.0,0.0,0.0,0.0,0.0,-1.32,-1.5875,-0.739409,-0.753846,0.0,...,0.677083,0.0,0.0,2023-02-13 11:00:00,MSP,DFW,Delta Air Lines Inc.,False,Not Cancelled,False
3,0.0,0.0,0.0,0.0,0.0,0.64,0.721371,0.216945,0.230769,0.0,...,-1.520833,0.0,0.0,2023-05-07 18:00:00,MIA,PNS,Envoy Air,False,Not Cancelled,False
4,4.2,36.0,0.0,0.0,0.0,-0.44,-0.439516,1.43389,1.723077,0.0,...,-2.319444,0.0,0.0,2023-03-18 16:00:00,DFW,DRT,SkyWest Airlines Inc.,False,Not Cancelled,True
5,0.1,0.0,0.0,0.0,0.0,0.92,0.985484,0.086008,-0.046154,0.0,...,-0.520833,0.0,0.0,2022-07-16 19:00:00,ATL,LAX,Spirit Air Lines,False,Not Cancelled,True
6,0.0,0.0,0.0,0.0,0.0,0.04,0.044355,-0.56611,-0.692308,-3218.68,...,-0.440972,0.0,0.0,2021-05-03 12:00:00,LAX,SMF,SkyWest Airlines Inc.,False,Not Cancelled,False
7,0.0,0.0,0.0,0.0,0.0,-0.44,-0.439516,0.129653,0.169231,0.0,...,0.277778,0.0,0.0,2022-05-02 14:00:00,DTW,ORD,SkyWest Airlines Inc.,False,Not Cancelled,False
8,0.0,0.0,0.0,0.0,0.0,-0.12,-0.112903,-2.0,-2.4,0.0,...,0.479167,0.0,0.0,2023-06-06 21:00:00,BOS,MCO,Spirit Air Lines,False,Not Cancelled,False
9,0.1,0.0,0.0,0.0,0.0,1.2,1.816129,0.086008,0.123077,0.0,...,-1.319444,0.0,0.0,2023-07-10 13:00:00,MIA,IAD,United Air Lines Inc.,False,Not Cancelled,True


In [ ]:
X_test_preproc.head()

,DepDelayMinutes,CarrierDelay,NASDelay,SecurityDelay,LateAircraftDelay,Temperature,Feels_Like_Temperature,Altimeter_Pressure,Sea_Level_Pressure,Visibility,...,Wind_Gust,Precipitation,Ice_Accretion_3hr,Time,Origin,Dest,Carrier,Cancelled,CancellationReason,Delayed
0,0.0,0.0,0.0,0.0,0.0,0.8,1.018548,0.086008,-0.061538,0.0,...,0.878472,0.0,0.0,2022-05-10 12:00:00,DFW,ATL,Delta Air Lines Inc.,False,Not Cancelled,False
1,0.0,0.0,0.0,0.0,0.0,0.48,0.487903,0.042362,-0.107692,0.0,...,-0.520833,0.0,0.0,2023-05-09 21:00:00,ATL,SJU,Spirit Air Lines,False,Not Cancelled,False
2,0.7,0.0,0.0,0.0,0.0,-0.08,-0.076613,-0.522465,-0.661538,-3218.68,...,-0.440972,0.0,0.0,2022-06-03 10:00:00,LAX,BOI,SkyWest Airlines Inc.,False,Not Cancelled,True
3,0.3,0.0,0.0,0.0,0.0,0.0,0.0,-0.956354,0.292308,0.0,...,-1.0,0.0,0.0,2023-03-03 11:00:00,CLT,OMA,American Airlines Inc.,False,Not Cancelled,True
4,1.2,0.0,0.0,0.0,0.0,0.72,0.914113,0.216945,0.307692,-4828.02,...,-1.0,0.0,0.0,2023-07-01 11:00:00,CLT,RDU,American Airlines Inc.,False,Not Cancelled,True


In [ ]:
X_train_preproc.columns

Index(['DepDelayMinutes', 'CarrierDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'Temperature', 'Feels_Like_Temperature',
       'Altimeter_Pressure', 'Sea_Level_Pressure', 'Visibility', 'Wind_Speed',
       'Wind_Gust', 'Precipitation', 'Ice_Accretion_3hr', 'Time', 'Origin',
       'Dest', 'Carrier', 'Cancelled', 'CancellationReason', 'Delayed'],
      dtype='object')

## Handling Encoding with OneHotEncoder

In [ ]:
ohe = OneHotEncoder()

X_train_cat_cols =X_train_preproc.select_dtypes(include=object)
X_test_cat_cols =X_test_preproc.select_dtypes(include=object)

X_train_encoded = pd.DataFrame(ohe.fit_transform(X_train_cat_cols), columns=X_train_cat_cols.columns)

X_test_encoded = pd.DataFrame(ohe.fit_transform(X_test_cat_cols), columns=X_test_cat_cols.columns)

NameError: name 'OneHotEncoder' is not defined

In [ ]:

X_train_encoded.head(1)
X_test_encoded.head(1)

# Exporting Cleaned Data to CSV within Data Directory


In [ ]:
data_directory = '../data/'

output_csv = 'preprocessed_with_outliers.csv'

output_path = os.path.join(data_directory, output_csv)

df_imputed.to_csv(output_path, header=True, index=False, compression='zip')

# Balancing

# Diego's preprocessed dataset